## Part 4 - Deploy Model

**Connect to Azure ML workspace**

In [37]:
from azureml.core import Workspace

workspace = Workspace.from_config()

**Get model**

In [38]:
from azureml.core import Model

model = Model(workspace, name='pokemon-classifier')

**Define inference config**

In [39]:
from azureml.core.model import InferenceConfig

inference_config = InferenceConfig(source_directory='scoring',
                                   runtime='python', 
                                   entry_script='score.py',
                                   conda_file='env.yml')

**Deploy web service**

In [65]:
from azureml.core.webservice import Webservice
from azureml.exceptions import WebserviceException
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)
aci_service_name = 'pokemon-classifier'

try:
    aci_service = Webservice(workspace, name=aci_service_name)
    if aci_service:
        aci_service.delete()
except WebserviceException as e:
    print()

aci_service = Model.deploy(workspace, aci_service_name, [model], inference_config, aci_config)
aci_service.wait_for_deployment(True)

Running...........................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


**Test web service**

In [66]:
service_url = aci_service.scoring_uri

In [182]:
# Load classes
classes = []
f = open('training/classes.txt','r')
for line in f.readlines():
    classes.append(line.replace('\n',''))
    f.close()

In [196]:
from ipywidgets import Layout, Button, Box, Label, Text, VBox
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO
from matplotlib import rcParams
%matplotlib inline

def on_button_clicked(b):
    with output:
        image_url = label.children[1].value
        
        # Make HTTP call
        input_data = '{\"image_url\": \"'+ image_url +'\"}'
        headers = {'Content-Type':'application/json'}
        response = requests.post(service_url, input_data, headers=headers)
        
        # Download image
        image = Image.open(BytesIO(requests.get(test_image).content))

        # Get prediction
        result = np.array(json.loads(json.loads(response.text))['prediction'][0])
        prediction = 'Prediction: {}\n Probability: {}'.format(classes[result.argmax()], result.max())

        # Display image
        plt.title(prediction)
        plt.imshow(image)
        plt.show()

label = Box([Label(value='Pokemon Image URL: '), Text()], layout=form_item_layout)
button = Button(description='Get Prediction!')
button.on_click(on_button_clicked)

form = Box([label, button], layout=Layout(
    display='flex',
    flex_flow='row',
    align_items='stretch',
    width='100%'
))

In [198]:
# Example: https://raw.githubusercontent.com/johnwu0604/techx-ml-pokedex-base/master/model/examples/pikachu_toy.png
VBox([form, output])

In [199]:
output.clear_output()